In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import pandas_profiling

# library untuk eksplorasi data
import matplotlib.pyplot as plt
import seaborn as sns

# library untuk membagi data
from sklearn.model_selection import train_test_split

# library untuk evaluasi hasil prediksi
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score

# library untuk bebrapa model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.feature_selection import SelectFromModel

# library untuk tunning parameter
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# library untuk menggabungkan beberapa algoritme
from imblearn.pipeline import Pipeline
# library untuk menyeimbangkan data
from imblearn.over_sampling import SMOTE

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
# import category_encoders as ce

from sklearn.calibration import CalibratedClassifierCV

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df_train = pd.read_csv('train_prep.csv')
df_test = pd.read_csv('test_prep.csv')

# Data Prepaparation

In [5]:
df_train_prep1 = df_train.copy()
df_test_prep1 = df_test.copy()

FE SKLEARN

In [6]:
new_var_one_hot = ['age2','age3','year_graduated2','in study','person level up','sick_vs_leaves']

one_hot_var = [
    'job_level_oh', 
    'person_level_oh', 
    'Employee_type', 
    'Education_level',
#     'Employee_status',   
    'gender', 
    'marital_status_maried(Y/N)'] + new_var_one_hot

ordinal_val = [
    'job_level',
    'person_level',
    'Education_level'
#     'achievement_target_1', 
#     'achievement_target_2'
]

new_var_num = ['year_graduated','total leave','total_rotation',
               'age when graduated','age when graduated2','papa muda',
               'papa muda per tanggungan','age per dependencies']

numeric_var = [
    'job_duration_in_current_job_level',   
    'job_duration_in_current_person_level',
    'job_duration_in_current_branch', 
    'age', 
    'number_of_dependences',
    'GPA',   
    'branch_rotation', 
    'job_rotation', 
    'assign_of_otherposition',   
    'annual leave', 
    'sick_leaves',
    'Last_achievement_%', 
    'Achievement_above_100%_during3quartal'] + new_var_num

In [7]:
# ordinal_mapping = [
#     {'col':'job_level',
#     'mapping':{None:0,'JG04':1,'JG05':2,'JG03':3,'JG06':4}},
#     {'col':'person_level',
#     'mapping':{None:0,'PG01':1,'PG02':2,'PG03':3,'PG04':4, 'PG05':5, 'PG06':6, 'PG07':7, 'PG08':8}},
#     {'col':'Education_level',
#     'mapping':{None:0,'level_0':1,'level_1':2,'level_2':3, 'level_3':4, 'level_4':5, 'level_5':6}}    
#     ]

# ordinal_encoder = ce.OrdinalEncoder(cols = ['job_level','person_level','Education_level'],mapping = ordinal_mapping)

one_hot_encoder_pipeline = Pipeline([
                                    ('one hot encoder',OneHotEncoder(handle_unknown = 'ignore'))
])

# numerical_pipeline = SimpleImputer(strategy = 'constant',fill_value = 0)

numerical_pipeline = Pipeline([
                                ('imputer',SimpleImputer(strategy = 'constant',fill_value = 0))
#                                 ('bin',KBinsDiscretizer(encode = 'onehot',strategy = 'kmeans',n_bins=5))
])

transformer = ColumnTransformer([
    ('one hot encoder',one_hot_encoder_pipeline,one_hot_var),
#     ('ordinal encoder',ordinal_encoder,ordinal_val),
    ('numerical_pipeline',numerical_pipeline,numeric_var)
])

In [8]:
# ?KBinsDiscretizer

# Data Splitting

In [9]:
var_x = numeric_var + one_hot_var #+ ordinal_val

X = df_train_prep1[var_x]
X_sub = df_test_prep1[var_x]
y = df_train_prep1['Best Performance']

In [ ]:
# seed = 1995
seed = 70621

X_trainval, X_test, y_trainval, y_test = train_test_split(X,y, stratify = y, test_size = 1500, random_state = seed)
X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, stratify = y_trainval, test_size = 1500, random_state = seed)

> # Model (Benchmark)

In [ ]:
rates = 0.2

model = XGBClassifier(
    learning_rate = rates,
    n_estimators = 100,
    max_depth = 5,
    min_child_weight = 6,
    gamma= 30,
    subsample = 0.7,
    colsample_bynode = 0.4,
    reg_alpha= 0.005,
    objective= 'binary:logistic',
    n_jobs = -1,
    random_state = 20)


# model = CalibratedClassifierCV(model)

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('clf',model)
])

In [ ]:
estimator.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('one hot encoder',
                                                  Pipeline(steps=[('one hot '
                                                                   'encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['job_level_oh',
                                                   'person_level_oh',
                                                   'Employee_type',
                                                   'Education_level', 'gender',
                                                   'marital_status_maried(Y/N)',
                                                   'age2', 'age3',
                                                   'year_graduated2',
                                                   'in study',
                                                   'person level up'

In [ ]:
y_prob_val = estimator.predict_proba(X_val)[:,1]
y_pred_val = np.where(y_prob_val > 0.13, 1, 0)

print('auc',roc_auc_score(y_val,y_prob_val),'\n')
print('classification report \n', classification_report(y_val,y_pred_val))

auc 0.5482350852272727 

classification report 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1280
           1       0.15      1.00      0.26       220

    accuracy                           0.15      1500
   macro avg       0.07      0.50      0.13      1500
weighted avg       0.02      0.15      0.04      1500



C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [ ]:
y_prob_train = estimator.predict_proba(X_train)[:,1]
y_pred_train = np.where(y_prob_train > 0.13, 1, 0)

print('auc',roc_auc_score(y_train,y_prob_train),'\n')
print('classification report \n', classification_report(y_train,y_pred_train))

auc 0.5622856499116298 

classification report 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      6956
           1       0.15      1.00      0.26      1197

    accuracy                           0.15      8153
   macro avg       0.07      0.50      0.13      8153
weighted avg       0.02      0.15      0.04      8153



C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [ ]:
y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

auc 0.5683096590909091 

classification report 
 

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1280
           1       0.15      1.00      0.26       220

    accuracy                           0.15      1500
   macro avg       0.07      0.50      0.13      1500
weighted avg       0.02      0.15      0.04      1500



> ## Hyperparameter Tuning (2nd Scenario)

1

In [ ]:
rates = 0.2

In [ ]:
model = XGBClassifier(
    learning_rate = rates,
    n_estimators = 100,
    max_depth = 5,
    min_child_weight = 6,
    gamma= 30,
    subsample = 0.7,
    colsample_bynode = 0.4,
    reg_alpha= 0.005,
    objective= 'binary:logistic',
    n_jobs = -1,
    random_state = 20)

# model = CalibratedClassifierCV(model)

select = SelectFromModel(model, threshold="median")

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 200)

hyperparam_space = {
    'clf__n_estimators':[60,80,100,120,150]
}

grid_search = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'roc_auc', # metrics
    n_jobs = -1 # use all cores
)

In [ ]:
grid_search.fit(X_trainval, y_trainval)

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
estimator = grid_search.best_estimator_
estimator.fit(X_trainval,y_trainval)

y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

2

In [ ]:
model = XGBClassifier(
    learning_rate = rates,
    n_estimators = grid_search.best_params_['clf__n_estimators'],
    max_depth = 5,
    min_child_weight = 6,
    gamma= 30,
    subsample = 0.7,
    colsample_bynode = 0.4,
    reg_alpha= 0.005,
    objective= 'binary:logistic',
    n_jobs = -1,
    random_state = 20)

# model = CalibratedClassifierCV(model)

select = SelectFromModel(model, threshold="median")

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

# skfold = StratifiedKFold(n_splits = 4)

hyperparam_space = {
    'clf__max_depth':range(3,6),
    'clf__min_child_weight':range(1,10)
}

grid_search2 = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'roc_auc', # metrics
    n_jobs = -1 # use all cores
)

In [ ]:
grid_search2.fit(X_trainval, y_trainval)

In [ ]:
print(grid_search2.best_params_)
print(grid_search2.best_score_)

In [ ]:
estimator = grid_search2.best_estimator_
estimator.fit(X_trainval,y_trainval)

y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

3

In [ ]:
model = XGBClassifier(
    learning_rate = rates,
    n_estimators = grid_search.best_params_['clf__n_estimators'],
    max_depth = grid_search2.best_params_['clf__max_depth'],
    min_child_weight = grid_search2.best_params_['clf__min_child_weight'],
    gamma= 30,
    subsample = 0.7,
    colsample_bynode = 0.4,
    reg_alpha= 0.005,
    objective= 'binary:logistic',
    n_jobs = -1,
    random_state = 20)

# model = CalibratedClassifierCV(model)

select = SelectFromModel(model, threshold="median")

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

# skfold = StratifiedKFold(n_splits = 4)

hyperparam_space = {
    'clf__gamma':[25,30,35,40],
    'clf__colsample_bynode':[i/10 for i in range(2,6)]
}

grid_search3 = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'roc_auc', # metrics
    n_jobs = -1 # use all cores
)

In [ ]:
grid_search3.fit(X_trainval, y_trainval)

In [ ]:
print(grid_search3.best_params_)
print(grid_search3.best_score_)

In [ ]:
estimator = grid_search3.best_estimator_
estimator.fit(X_trainval,y_trainval)

y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

4

In [ ]:
model = XGBClassifier(
    learning_rate = rates,
    n_estimators = grid_search.best_params_['clf__n_estimators'],
    max_depth = grid_search2.best_params_['clf__max_depth'],
    min_child_weight = grid_search2.best_params_['clf__min_child_weight'],
    gamma= grid_search3.best_params_['clf__gamma'],
    subsample = 0.7,
    colsample_bynode = grid_search3.best_params_['clf__colsample_bynode'],
    reg_alpha= 0.005,
    objective= 'binary:logistic',
    n_jobs = -1,
    random_state = 20)

# model = CalibratedClassifierCV(model)

select = SelectFromModel(model, threshold="median")

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

# skfold = StratifiedKFold(n_splits = 4)

hyperparam_space = {
    'clf__subsample':[i/10.0 for i in range(6,8)]
}

grid_search4 = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'roc_auc', # metrics
    n_jobs = -1 # use all cores
)

In [ ]:
grid_search4.fit(X_trainval, y_trainval)

In [ ]:
print(grid_search4.best_params_)
print(grid_search4.best_score_)

In [ ]:
estimator = grid_search4.best_estimator_
estimator.fit(X_trainval,y_trainval)

y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

5

In [ ]:
model = XGBClassifier(
    learning_rate = rates,
    n_estimators = grid_search.best_params_['clf__n_estimators'],
    max_depth = grid_search2.best_params_['clf__max_depth'],
    min_child_weight = grid_search2.best_params_['clf__min_child_weight'],
    gamma= grid_search3.best_params_['clf__gamma'],
    subsample = grid_search4.best_params_['clf__subsample'],
    colsample_bynode = grid_search3.best_params_['clf__colsample_bynode'],
    reg_alpha= 0.005,
    objective= 'binary:logistic',
    n_jobs = -1,
    random_state = 20)

# model = CalibratedClassifierCV(model)

select = SelectFromModel(model, threshold="median")

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

# skfold = StratifiedKFold(n_splits = 4)

hyperparam_space = {
    'clf__reg_alpha':[0.0005, 0.005, 0.001, 0.05, 0.01, 0.5, 0.1, 10],
    'clf__reg_lambda':[None, 0.001, 0.01, 0.1, 1, 10]
}

grid_search5 = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'roc_auc', # metrics
    n_jobs = -1 # use all cores
)

In [ ]:
grid_search5.fit(X_trainval, y_trainval)

In [ ]:
print(grid_search5.best_params_)
print(grid_search5.best_score_)

In [ ]:
estimator = grid_search5.best_estimator_
estimator.fit(X_trainval,y_trainval)

y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

6

In [ ]:
model = XGBClassifier(
    learning_rate = rates,
    n_estimators = grid_search.best_params_['clf__n_estimators'],
    max_depth = grid_search2.best_params_['clf__max_depth'],
    min_child_weight = grid_search2.best_params_['clf__min_child_weight'],
    gamma= grid_search3.best_params_['clf__gamma'],
    subsample = grid_search4.best_params_['clf__subsample'],
    colsample_bynode = grid_search3.best_params_['clf__colsample_bynode'],
    reg_alpha = grid_search5.best_params_['clf__reg_alpha'],
    reg_lambda = grid_search5.best_params_['clf__reg_lambda'],
    objective = 'binary:logistic',
    n_jobs = -1)

# model = CalibratedClassifierCV(model)

select = SelectFromModel(model, threshold="median")

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

# skfold = StratifiedKFold(n_splits = 4)

hyperparam_space = [
    {'clf__learning_rate':[rates],'clf__n_estimators':[grid_search.best_params_['clf__n_estimators']]},
    {'clf__learning_rate':[rates/2],'clf__n_estimators':[grid_search.best_params_['clf__n_estimators']*2]}#,
#     {'clf__learning_rate':[rates/4],'clf__n_estimators':[grid_search.best_params_['clf__n_estimators']*4]},
#     {'clf__learning_rate':[rates/8],'clf__n_estimators':[grid_search.best_params_['clf__n_estimators']*8]},
#     {'clf__learning_rate':[rates/16],'clf__n_estimators':[grid_search.best_params_['clf__n_estimators']*16]}
]

grid_search6 = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'roc_auc', # metrics
    n_jobs = -1 # use all cores
)

In [ ]:
grid_search6.fit(X_trainval, y_trainval)

In [ ]:
print(grid_search6.best_params_)
print(grid_search6.best_score_)

In [ ]:
estimator = grid_search6.best_estimator_
estimator.fit(X_trainval,y_trainval)

y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

In [ ]:
model = XGBClassifier(
    learning_rate = grid_search6.best_params_['clf__learning_rate'],
    n_estimators = grid_search6.best_params_['clf__n_estimators'],
    max_depth = grid_search2.best_params_['clf__max_depth'],
    min_child_weight = grid_search2.best_params_['clf__min_child_weight'],
    gamma= grid_search3.best_params_['clf__gamma'],
    subsample = grid_search4.best_params_['clf__subsample'],
    colsample_bynode = grid_search3.best_params_['clf__colsample_bynode'],
    reg_alpha = grid_search5.best_params_['clf__reg_alpha'],
    reg_lambda = grid_search5.best_params_['clf__reg_lambda'],
    objective = 'binary:logistic',
    n_jobs = -1)

select = SelectFromModel(model, threshold="median")

model = CalibratedClassifierCV(model)

final_estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

In [ ]:
final_estimator.fit(X_trainval,y_trainval)

y_prob_test = final_estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

In [ ]:
y_prob_trainval = final_estimator.predict_proba(X_trainval)[:,1]
y_pred_trainval = np.where(y_prob_trainval > 0.13, 1, 0)

print('auc',roc_auc_score(y_trainval,y_prob_trainval),'\n')
print('classification report \n', classification_report(y_trainval,y_pred_trainval))

# Predict

In [ ]:
model = XGBClassifier(
    learning_rate = grid_search6.best_params_['clf__learning_rate'],
    n_estimators = grid_search6.best_params_['clf__n_estimators'],
    max_depth = grid_search2.best_params_['clf__max_depth'],
    min_child_weight = grid_search2.best_params_['clf__min_child_weight'],
    gamma= grid_search3.best_params_['clf__gamma'],
    subsample = grid_search4.best_params_['clf__subsample'],
    colsample_bynode = grid_search3.best_params_['clf__colsample_bynode'],
    reg_alpha = grid_search5.best_params_['clf__reg_alpha'],
    reg_lambda = grid_search5.best_params_['clf__reg_lambda'],
    objective = 'binary:logistic',
    n_jobs = -1)

select = SelectFromModel(model, threshold="median")

model = CalibratedClassifierCV(model)

final_estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

In [ ]:
# final_estimator = grid_search2.best_estimator_

In [ ]:
final_estimator.fit(X, y)

In [ ]:
y_pred_sub = final_estimator.predict_proba(X_sub)[:,1]

In [ ]:
df_test['Best Performance'] = y_pred_sub

In [ ]:
df_submission = df_test[['Best Performance']]

In [ ]:
df_submission.reset_index().to_csv('submission v1 - step by step.csv',index = False)